<a href="https://colab.research.google.com/github/Skyrub-dev/BT_AI/blob/main/BT_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **L6 End-of-year project**

**Import Files**

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/data')

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Alternatively, through a local machine**

**Importing libraries**

---



In [ ]:
import os
import tensorflow as tf
import cv2 #checkout open cv
import imghdr #allows to check file extensions for images
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
#import kaggle - will likely change

from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import sequential
from keras.preprocessing.image import ImageDataGenerator

GPU check optimisation

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

CPU / GPU debug

In [ ]:
tf.config.list_physical_devices(device_type=None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

**If running off a local machine, this line of code must be run to install all dependencies**

---



In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib #! || % at the beginning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.5 MB/s eta 0:00:00


**Importing and loading the data**

---



To import our data, we're going to be using an approach known as "Supervised learning". This is defined through pre-labelled datasets which the AI will be trained on. This is the best approach to solve a classification problem as the model can adjust itself and learn from its mistakes.

https://www.youtube.com/watch?v=_L2uYfVV48I

This is something that you do only for toy datasets or small sample sets. In the reality using image data generator is not recommended when you have huge volumes of images or video frames (which is usually the case). In those cases is a good practice to load and split the data across multiple tfrecords, then loading it progressively with tf.data and perform multithreaded augmentations dynamically during training. The majority of the augmentations can be computed using low level tf or common python functions.

In [ ]:
labels = ['giloma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

Loading the images

In [ ]:
os.listdir('/content/drive/MyDrive/data/archive/Testing')

['glioma_tumor', 'pituitary_tumor', 'no_tumor', 'meningioma_tumor']

In [ ]:
#https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/

testing_path = '/content/drive/MyDrive/data/archive/Testing'
training_path = '/content/drive/MyDrive/data/archive/Training'
validation_path = '/content/drive/MyDrive/data/archive/Validation'

img_size = 150 #Likely not necessary but could be used

classes = 4
#Normalising pixel values to be between 0 and 1
testing_data_generator = ImageDataGenerator(rescale = 1./255)
training_data_generator = ImageDataGenerator(rescale = 1./255)
validation_data_generator = ImageDataGenerator(rescale = 1./255)

testing_generator = testing_data_generator.flow_from_directory (testing_path,
                                                                target_size = (150, 150),
                                                                color_mode = 'rgb',
                                                                save_format = 'jpg',
                                                                batch_size = 32,
                                                                class_mode = 'categorical',
                                                                classes = ['giloma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'])

#Need to check these two and make sure they're configured correctly

training_generator = training_data_generator.flow_from_directory (training_path,
                                                                target_size = (150, 150),
                                                                color_mode = 'rgb',
                                                                save_format = 'jpg',
                                                                batch_size = 32,
                                                                class_mode = 'categorical',
                                                                classes = ['giloma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'])

validation_generator = testing_data_generator.flow_from_directory (validation_path,
                                                                target_size = (150, 150),
                                                                color_mode = 'rgb',
                                                                save_format = 'jpg',
                                                                batch_size = 32,
                                                                class_mode = 'categorical',
                                                                classes = ['giloma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'])


Found 294 images belonging to 4 classes.
Found 2044 images belonging to 4 classes.
Found 0 images belonging to 4 classes.


Test code below, can delete or use as backup if things go wrong

In [ ]:
training = []
training_label = []

testing = []
testing_label = []

#test_path = os.listdir('/content/drive/MyDrive/data/archive/Training')

for i in testing_path:
  image = cv2.imread(testing_path, cv2.IMREAD_COLOR)
  image = cv2.resize(image, (150, 150))
  image = image / 255.0 #normalisation if above chunk of code works, will likely not need to apply this
  testing.append(image)
  testing_label.append(labels)

error: ignored

Next I'll likely use Keras’ ImageDataGenerator function to augment the dataset

In [ ]:
#Example: https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/
#src_path_train = "data/train/"
#src_path_test = "data/test/"

#adjust this function

data_agument = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.05,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    shear_range = 0.05,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.20
)

#train_datagen = ImageDataGenerator(
        #rescale=1 / 255.0,
        #rotation_range=20,
        #zoom_range=0.05,
        #width_shift_range=0.05,
        #height_shift_range=0.05,
        #shear_range=0.05,
        #horizontal_flip=True,
        #fill_mode="nearest",
        #validation_split=0.20)

#test_datagen = ImageDataGenerator(rescale=1 / 255.0)